In [4]:
import csv
import re
import pickle
import pandas as pd

def unescape(s):
    """
        Unescape an sql escaped dump string
    """
    to_sub=re.escape(",'\\\n")
    return re.sub('\\\([%s])'%(to_sub,), r'\g<1>', s).replace(r'\0','\0')
        
def read_embedding_file(filename, records_number):
    ''' return a dictinoray {'id': embedding} where each embedding is itself 
        a panda series
        Inputs:
            filename: embedding file name
            records_number: number of the records to be read                
    '''
    embeddings={}
    with open(filename, 'rb') as f:
        freader = csv.reader(f, delimiter=',', quotechar="'")
        for i in range(records_number):
            row = freader.next()
            embeddings[row[0]]= unescape(row[1])
            #values, index = pickle.loads(unescape(row[1]))
            #embeddings[row[0]]=pd.Series(values, index=index)
    return embeddings
    

In [21]:
embeddings={}

with open('/users/grad/sajadi/backup/tmp/pagelinkstest1.tsv', 'rb') as f:
    freader = csv.reader(f, delimiter=',', quotechar="'")
    
    i=0
    for row in freader:
        embeddings[row[0]]= unescape(row[1])
        i += 1
        #values, index = pickle.loads(unescape(row[1]))
        #embeddings[row[0]]=pd.Series(values, index=index)


Error: new-line character seen in unquoted field - do you need to open the file in universal-newline mode?

In [34]:
filename = '/users/grad/sajadi/backup/tmp/pagelinksorderedout.csv'
embeddings = read_embedding_file (filename,5)

Error: new-line character seen in unquoted field - do you need to open the file in universal-newline mode?

In [20]:
import MySQLdb
tablename = 'pagelinksorderedout';
colname = 'out_neighb'

_db = MySQLdb.connect(host="127.0.0.1",port=3307,user='root',passwd="emilios",db="enwiki20160305")
_cursor = _db.cursor()
db_embeddings ={}
#tablename = 'pagelinksorderedout';
#colname = 'out_neighb';
for wid in embeddings.keys():
    query =    """select {0} from {1} where cache_id={2}""".format(colname, tablename, wid)
    _cursor.execute(query);
    row = _cursor.fetchone();
    db_embeddings[wid] = row[0]

#     values, index = pickle.loads(row[0])
#     em=pd.Series(values, index=index)
#     db_embeddings[wid] = em
    

In [22]:
embeddings

{'3632887': '\x80\x02]q\x01(G?\xd4#0\xdb\xca3WG?\x80+\x1d\xa4a\x02\xb3G?\x8aeX\xe1k\xa5QG?\xc7\xa8\x19\x02\x90m\x86G?\x8aeX\xe1k\xa5QG?\x80+\x1d\xa4a\x02\xb3G?\x80+\x1d\xa4a\x02\xb3G?\xae\xeb:\x9e6#YG?\xaaE\xafH\xcb\xb8nG?\x84\xbf\xdd\x81\xe2\xe5\x99G?\x80+\x1d\xa4a\x02\xb3G?\x9bU\xcd)\xf2\r{G?\x80+\x1d\xa4a\x02\xb3G?\x87\n=p\xa3\xd7\x0cG?\x80+\x1d\xa4a\x02\xb3G?\xb6\xaeW\xf2?\xe0<G?\x80+\x1d\xa4a\x02\xb3G?\x9er\t\x0c\x17\xc9\x8aG?\xc2\x18\xfaH\xf988e]q\x02(J\xf7n7\x00M\xcaIM\xf2XM\xe6\xfbJ\x91\xa5\x02\x00J\xb9\xcb\x05\x00JH\xbd\x07\x00J\xa8*\t\x00J\xff\x91\n\x00JQ\xbb\x11\x00J\x03\xec\x12\x00J\x9ey\x19\x00J\x1bI\x1d\x00J\x95y9\x00J3\xf2J\x00J\xf2\xafu\x00J\xbe*\xa2\x01JX\xe4\xd6\x01Jz\x1bR\x02e\x86.'}

In [23]:
embeddings == db_embeddings

True

In [33]:
A = {3:5, 6:4, 7:1, -11:1}
B = {7:1, -11:1, 6:4, 3:5}

{-11: 1, 3: 5, 6: 4, 7: 1}